<a href="https://colab.research.google.com/github/PetrovES570/PDA_SOL/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%91%D0%B8%D0%BB%D0%B5%D1%8215%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install duckdb jupysql duckdb-engine --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install altair

In [ ]:
# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%config SqlMagic.autocommit=False  # Для автоматического выполнения

In [ ]:
import duckdb
import pandas as pd
import altair as alt

# Create DuckDB connection
con = duckdb.connect()

In [ ]:
duckdb.__version__

'1.1.3'

In [ ]:
%sql con

Если нет подключения к Postgres, можно загрузить данные привычным споособом:

In [ ]:
#df = pd.read_csv('/content/US_Regional_Sales_Data.csv', delimiter=',', encoding='unicode_escape')

In [ ]:
pip install squarify

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import squarify
from operator import attrgetter
import requests
import warnings
warnings.filterwarnings("ignore")
from urllib.parse import urlencode
%matplotlib inline

In [ ]:
%%sql
CREATE TABLE SalesData AS
SELECT DISTINCT
    OrderNumber,
        "Sales Channel",
        WarehouseCode,
        ProcuredDate,
        OrderDate,
        ShipDate,
        DeliveryDate,
        _SalesTeamID,
        _CustomerID,
        _StoreID,
        _ProductID,
        "Order Quantity",
        "Discount Applied",
        "Unit Cost",
        "Unit Price",
FROM read_csv_auto('/content/US_Regional_Sales_Data.csv', ignore_errors=true)
WHERE _CustomerID IS NOT NULL

,Count
0,7991


In [ ]:
%%sql
UPDATE SalesData
SET "Unit Price" = CAST(REPLACE("Unit Price", ',', '') AS INTEGER);


,Count
0,7991


In [ ]:
%%sql
select * from SalesData

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price
0,SO - 000101,In-Store,WARE-UHY1004,2031-12-17,2031-05-18,2014-06-18,2019-06-18,6,15,259,12,5,0.075,"1,001.18",1963
1,SO - 000102,Online,WARE-NMK1003,2031-12-17,2031-05-18,2022-06-18,2002-07-18,14,20,196,27,3,0.075,"3,348.66",3940
2,SO - 000103,Distributor,WARE-UHY1004,2031-12-17,2031-05-18,2021-06-18,2001-07-18,21,16,213,16,1,0.050,781.22,1776
3,SO - 000106,Online,WARE-PUJ1005,2031-12-17,2031-05-18,2008-06-18,2013-06-18,12,21,285,1,5,0.050,446.56,1039
4,SO - 000107,In-Store,WARE-XYS1001,2031-12-17,2031-05-18,2008-06-18,2014-06-18,10,14,6,5,4,0.150,536.67,1193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7986,SO - 0008082,Distributor,WARE-PUJ1005,2026-09-20,2029-12-20,2007-01-21,2011-01-21,22,12,279,17,1,0.100,"1,303.42",1836
7987,SO - 0008086,In-Store,WARE-NBV1002,2026-09-20,2029-12-20,2009-01-21,2011-01-21,5,27,61,26,6,0.075,137.48,255
7988,SO - 0008088,Online,WARE-NMK1003,2026-09-20,2030-12-20,2002-01-21,2004-01-21,14,29,202,3,6,0.050,"1,921.56",3203
7989,SO - 0008089,Online,WARE-UHY1004,2026-09-20,2030-12-20,2023-01-21,2026-01-21,14,32,241,35,5,0.200,"2,792.76",3826


In [ ]:
%%sql
WITH ABCXYZ_Analysis AS (
    SELECT
        _ProductID,
        SUM(CAST("Order Quantity" AS INTEGER) * CAST(REPLACE("Unit Price", ',', '') AS INTEGER)) AS TotalRevenue,
        STDDEV(CAST("Order Quantity" AS INTEGER)) AS OrderStdDev,
        AVG(CAST("Order Quantity" AS INTEGER)) AS OrderAvg,
        -- ABC классификация
        CASE
            WHEN SUM(CAST("Order Quantity" AS INTEGER) * CAST(REPLACE("Unit Price", ',', '') AS INTEGER)) >= 0.8 * SUM(SUM(CAST("Order Quantity" AS INTEGER) * CAST(REPLACE("Unit Price", ',', '') AS INTEGER))) OVER ()
            THEN 'A'
            WHEN SUM(CAST("Order Quantity" AS INTEGER) * CAST(REPLACE("Unit Price", ',', '') AS INTEGER)) >= 0.5 * SUM(SUM(CAST("Order Quantity" AS INTEGER) * CAST(REPLACE("Unit Price", ',', '') AS INTEGER))) OVER ()
            THEN 'B'
            ELSE 'C'
        END AS ABC_Class,
        -- XYZ классификация
        CASE
            WHEN (STDDEV(CAST("Order Quantity" AS INTEGER)) / NULLIF(AVG(CAST("Order Quantity" AS INTEGER)), 0)) * 100 <= 10 THEN 'X'
            WHEN (STDDEV(CAST("Order Quantity" AS INTEGER)) / NULLIF(AVG(CAST("Order Quantity" AS INTEGER)), 0)) * 100 <= 25 THEN 'Y'
            ELSE 'Z'
        END AS XYZ_Class
    FROM SalesData
    GROUP BY _ProductID
),
RFM_Analysis AS (
    SELECT
        _CustomerID,
        DATEDIFF('day', MAX(OrderDate), CURRENT_DATE) AS Recency, -- Разница в днях с последнего заказа
        COUNT(DISTINCT OrderNumber) AS Frequency, -- Количество уникальных заказов
        SUM(CAST("Order Quantity" AS INTEGER) * CAST(REPLACE("Unit Price", ',', '') AS INTEGER) - "Discount Applied") AS Monetary, -- Общая сумма продаж с учётом скидки
        -- Присвоение R, F, M категорий
        NTILE(5) OVER (ORDER BY DATEDIFF('day', MAX(OrderDate), CURRENT_DATE)) AS R_Score,
        NTILE(5) OVER (ORDER BY COUNT(DISTINCT OrderNumber)) AS F_Score,
        NTILE(5) OVER (ORDER BY SUM(CAST("Order Quantity" AS INTEGER) * CAST(REPLACE("Unit Price", ',', '') AS INTEGER) - "Discount Applied")) AS M_Score
    FROM SalesData
    GROUP BY _CustomerID
),
Final_Analysis AS (
    SELECT
        r._CustomerID,
        r.Recency,
        r.Frequency,
        r.Monetary,
        r.R_Score,
        r.F_Score,
        r.M_Score,
        -- Определение сегмента RFM
        CASE
            WHEN R_Score >= 4 AND F_Score >= 4 AND M_Score >= 4 THEN 'Champions'
            WHEN R_Score >= 3 AND F_Score >= 3 AND M_Score >= 3 THEN 'Loyal Customers'
            ELSE 'At Risk'
        END AS RFM_Segment,
        a._ProductID,
        a.TotalRevenue,
        a.ABC_Class || a.XYZ_Class AS ABCXYZ_Segment
    FROM RFM_Analysis r
    LEFT JOIN ABCXYZ_Analysis a ON a._ProductID = r._CustomerID -- Используем _ProductID для соединения
)
SELECT *
FROM Final_Analysis
ORDER BY RFM_Segment, ABCXYZ_Segment;


,_CustomerID,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Segment,_ProductID,TotalRevenue,ABCXYZ_Segment
0,1,-2516,152,1322293.650,1,2,1,At Risk,1.0,1436721.0,CZ
1,35,-2364,145,1555078.750,4,1,2,At Risk,35.0,1982013.0,CZ
2,46,-2303,157,1508126.075,5,3,2,At Risk,46.0,1837543.0,CZ
3,45,-2303,156,1630027.300,5,2,3,At Risk,45.0,1710019.0,CZ
4,5,-2516,159,1609240.225,1,3,3,At Risk,5.0,2011381.0,CZ
5,6,-2302,143,1530792.575,5,1,2,At Risk,6.0,1422355.0,CZ
6,7,-2516,153,1544097.825,1,2,2,At Risk,7.0,1727137.0,CZ
7,8,-2395,142,1411533.150,4,1,1,At Risk,8.0,1976953.0,CZ
8,44,-2516,156,1530037.450,1,3,2,At Risk,44.0,1268980.0,CZ
9,43,-2454,151,1564268.200,4,2,3,At Risk,43.0,1754789.0,CZ


In [ ]:
%%sql
-- Создание новой таблицы с анализом ABC, XYZ и RFM
CREATE TABLE SalesData_with_Analysis AS
WITH ABCXYZ_Analysis AS (
    SELECT
        _ProductID,
        SUM(CAST("Order Quantity" AS INTEGER) * CAST(REPLACE("Unit Price", ',', '') AS INTEGER)) AS TotalRevenue,
        STDDEV(CAST("Order Quantity" AS INTEGER)) AS OrderStdDev,
        AVG(CAST("Order Quantity" AS INTEGER)) AS OrderAvg,
        -- ABC классификация
        CASE
            WHEN SUM(CAST("Order Quantity" AS INTEGER) * CAST(REPLACE("Unit Price", ',', '') AS INTEGER)) >= 0.8 * SUM(SUM(CAST("Order Quantity" AS INTEGER) * CAST(REPLACE("Unit Price", ',', '') AS INTEGER))) OVER ()
            THEN 'A'
            WHEN SUM(CAST("Order Quantity" AS INTEGER) * CAST(REPLACE("Unit Price", ',', '') AS INTEGER)) >= 0.5 * SUM(SUM(CAST("Order Quantity" AS INTEGER) * CAST(REPLACE("Unit Price", ',', '') AS INTEGER))) OVER ()
            THEN 'B'
            ELSE 'C'
        END AS ABC_Class,
        -- XYZ классификация
        CASE
            WHEN (STDDEV(CAST("Order Quantity" AS INTEGER)) / NULLIF(AVG(CAST("Order Quantity" AS INTEGER)), 0)) * 100 <= 10 THEN 'X'
            WHEN (STDDEV(CAST("Order Quantity" AS INTEGER)) / NULLIF(AVG(CAST("Order Quantity" AS INTEGER)), 0)) * 100 <= 25 THEN 'Y'
            ELSE 'Z'
        END AS XYZ_Class
    FROM SalesData
    GROUP BY _ProductID
),
RFM_Analysis AS (
    SELECT
        _CustomerID,
        DATEDIFF('day', MAX(OrderDate), CURRENT_DATE) AS Recency, -- Разница в днях с последнего заказа
        COUNT(DISTINCT OrderNumber) AS Frequency, -- Количество уникальных заказов
        SUM(CAST("Order Quantity" AS INTEGER) * CAST(REPLACE("Unit Price", ',', '') AS INTEGER) - "Discount Applied") AS Monetary, -- Общая сумма продаж с учётом скидки
        -- Присвоение R, F, M категорий
        NTILE(5) OVER (ORDER BY DATEDIFF('day', MAX(OrderDate), CURRENT_DATE)) AS R_Score,
        NTILE(5) OVER (ORDER BY COUNT(DISTINCT OrderNumber)) AS F_Score,
        NTILE(5) OVER (ORDER BY SUM(CAST("Order Quantity" AS INTEGER) * CAST(REPLACE("Unit Price", ',', '') AS INTEGER) - "Discount Applied")) AS M_Score
    FROM SalesData
    GROUP BY _CustomerID
),
Final_Analysis AS (
    SELECT
        r._CustomerID,
        r.Recency,
        r.Frequency,
        r.Monetary,
        r.R_Score,
        r.F_Score,
        r.M_Score,
        -- Определение сегмента RFM
        CASE
            WHEN R_Score >= 4 AND F_Score >= 4 AND M_Score >= 4 THEN 'Champions'
            WHEN R_Score >= 3 AND F_Score >= 3 AND M_Score >= 3 THEN 'Loyal Customers'
            ELSE 'At Risk'
        END AS RFM_Segment,
        a._ProductID,
        a.TotalRevenue,
        a.ABC_Class || a.XYZ_Class AS ABCXYZ_Segment
    FROM RFM_Analysis r
    LEFT JOIN ABCXYZ_Analysis a ON a._ProductID = r._CustomerID -- Используем _ProductID для соединения
)
-- Вставляем данные в новую таблицу
SELECT
    s.*,  -- Все столбцы из таблицы SalesData
    f.Recency,
    f.Frequency,
    f.Monetary,
    f.RFM_Segment,
    f.ABCXYZ_Segment
INTO SalesData_with_Analysis -- Создание таблицы с анализом
FROM SalesData s
LEFT JOIN Final_Analysis f ON s._CustomerID = f._CustomerID  -- Используем _CustomerID для соединения с анализом
ORDER BY f.RFM_Segment, f.ABCXYZ_Segment;

-- Выгрузка данных в CSV
COPY SalesData_with_Analysis TO 'exported_data.csv' WITH (FORMAT CSV, HEADER TRUE);




,Count
0,7991


In [ ]:
%%sql
SELECT * FROM SalesData_with_Analysis where RFM_Segment = 'At Risk'

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price,Recency,Frequency,Monetary,RFM_Segment,ABCXYZ_Segment
0,SO - 0008058,Online,WARE-NMK1003,2026-09-20,2027-12-20,2008-01-21,2013-01-21,15,16,181,38,4,0.10,651.78,1018,-2362,135,1547804.025,At Risk,CZ
1,SO - 0002985,In-Store,WARE-UHY1004,2004-02-19,2004-05-19,2009-05-19,2012-05-19,5,19,230,46,6,0.10,"1,027.11",2446,-2515,165,1723563.875,At Risk,CZ
2,SO - 0004097,Wholesale,WARE-XYS1001,2015-05-19,2015-09-19,2028-09-19,2006-10-19,27,38,40,21,6,0.20,645.34,1152,-2060,150,1414008.425,At Risk,CZ
3,SO - 0004099,Wholesale,WARE-UHY1004,2015-05-19,2016-09-19,2021-09-19,2026-09-19,27,28,244,21,1,0.05,"2,465.60",3853,-2363,145,1408606.550,At Risk,CZ
4,SO - 000107,In-Store,WARE-XYS1001,2031-12-17,2031-05-18,2008-06-18,2014-06-18,10,14,6,5,4,0.15,536.67,1193,-2393,157,1452237.450,At Risk,CZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5777,SO - 0001344,Online,WARE-XYS1001,2019-07-18,2027-10-18,2030-10-18,2003-11-18,19,49,7,36,2,0.10,"1,294.64",2271,-2302,152,1503479.125,At Risk,None
5778,SO - 0001555,Online,WARE-NMK1003,2019-07-18,2021-11-18,2017-12-18,2023-12-18,15,49,138,30,3,0.05,722.93,871,-2302,152,1503479.125,At Risk,None
5779,SO - 0004175,Wholesale,WARE-PUJ1005,2015-05-19,2026-09-19,2002-10-19,2005-10-19,27,49,299,6,1,0.15,137.48,255,-2302,152,1503479.125,At Risk,None
5780,SO - 0002038,Online,WARE-MKL1006,2027-10-18,2009-01-19,2023-01-19,2028-01-19,15,49,359,30,3,0.15,481.19,891,-2302,152,1503479.125,At Risk,None
